In [1]:
pip install -q --upgrade langchain
pip install -q --upgrade langchain-google-genai 
pip install -q --upgrade google-generativeai  
pip install -q --upgrade langchain-core 
pip install -q --upgrade langchain_community 
pip install -q --upgrade docx2txt 
pip install -q --upgrade pypdf 
pip install -q --upgrade langchain_chroma
pip install -q --upgrade chromadb
pip install -q --upgrade sentence_transformers

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
#!pip uninstall -y google-generativeai google-ai-generativelanguage langchain-google-genai

In [3]:
#!pip install -q --upgrade langchain langchain-core langchain-community
#!pip install -q --upgrade langchain-google-genai google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15

In [3]:
import os
import langchain
from dotenv import load_dotenv
print(langchain.__version__)

0.3.27


In [5]:
#Calling LLM inside the langchain

In [6]:
#import google.generativeai as genai
#genai.configure(api_key=os.environ["GEMINI_API_KEY"])
#llm = genai.GenerativeModel("gemini-1.5-flash")
#llm_response = llm.generate_content("Hello")
#print(llm_response.text)

#We dont use this as it is raw Gemini SDK but we need Langchain wrappers

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv

# Load .env file from the current directory
load_dotenv()

# Fetch keys
gemini_key = os.getenv("GOOGLE_API_KEY")
langchain_key = os.getenv("LANGCHAIN_API_KEY")

# Check if keys were loaded correctly
if not gemini_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file")
if not langchain_key:
    raise ValueError("LANGCHAIN_API_KEY not found in .env file")

# Set environment variables
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["LANGCHAIN_API_KEY"] = langchain_key
os.environ["LANGCHAIN_PROJECT"] = "langchain_course"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Test call
llm_response = llm.invoke("Hello, what is RAG? Explain in 1 line")
print(llm_response.content)

RAG (Retrieval Augmented Generation) retrieves external information to augment a Large Language Model's (LLM) generation, providing more accurate and relevant answers.


In [25]:
#Model works 
#This also creates LangSmith Tracing course which we need. This couldn't be created using raw Gemini SDK, Therefore we need Langchain Wrappers 

GOOGLE_API_KEY: None
LANGCHAIN_API_KEY: None


In [9]:
#Parsing Output #AI Message in openai is different, and from that we need only plain text, so we have to parse
from langchain_core.output_parsers import StrOutputParser
output_parsers = StrOutputParser()
output_parsers.invoke(llm_response)

"RAG retrieves relevant external information to ground and improve an LLM's answer, making it more accurate and less prone to hallucination."

In [10]:
#Simple Chain
chain = llm | output_parsers
chain.invoke("Tell me a joke")

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [11]:
#To create a structured output from plain text
from typing import List
from pydantic import BaseModel, Field
class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of phone")
    rating: float = Field(description="Overall rating of the phone")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the reivew")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.
Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.
Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
print(output)

phone_model='Galaxy S21' rating=4.0 pros=['Gorgeous screen', 'Amazing camera, especially at night', 'Solid battery life'] cons=['Pricey', 'No charger included', 'New button layout takes getting used to (Bixby button issues)'] summary='Great phone with a few annoying quirks, but definitely worth checking out for an upgrade.'


In [12]:
#We create prompt templates to make the model i/p dynamic
#Till now we used static input, now using prompt template we will make it dynamic
#Here we use LCEL
#Instead of writing a plain string, you define a template with placeholders.
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}") #This creates a reusable prompt template
chain = prompt | llm | output_parsers #we create prompt first, which is sent to llm which is then parsed
result = chain.invoke({"topic": "programming"})
print(result)

#chain = prompt → LLM → parser

Here's one that's both historical and literal:

You know the term "bug" in programming, right? It refers to an error or flaw in code. Well, the term comes from an **actual bug!**

On **September 9, 1947**, computer pioneer Grace Hopper and her team at Harvard were working on the Mark II Aiken Relay Calculator. They couldn't figure out why the machine was malfunctioning. After much searching, they found a **moth** trapped in one of the relays, causing the error.

They carefully removed the moth and taped it into their logbook, with the note: **"First actual case of bug being found."**

This literal event is the origin of the terms "bug" for a computer error and "debugging" for the process of finding and fixing those errors!


In [13]:
#We can use LLM Messages to pass a list of messages
from langchain_core.messages import HumanMessage, SystemMessage
system_messages = SystemMessage(content="You are a helpful assistant that provides interesting facts")
human_messages = HumanMessage(content = "Tell me about programming")
llm.invoke([system_messages,human_messages])

AIMessage(content='Programming is a fascinating field that underpins almost everything we do in the digital world! Here are some interesting facts about it:\n\n1.  **The World\'s First Computer Programmer was a Woman:** Ada Lovelace, daughter of the poet Lord Byron, is often credited with writing the first algorithm intended to be carried out by Charles Babbage\'s Analytical Engine in the mid-19th century.\n2.  **The Original "Computers" Were Human:** Before electronic computers, "computers" were often women who performed complex mathematical calculations by hand, especially during wartime for ballistics trajectories. When the ENIAC (one of the first electronic computers) was built, these women were the ones who learned to program it by wiring switches and cables.\n3.  **"Bugs" Come from a Real Moth:** The term "bug" for an error in code originated in 1947 when computer pioneer Grace Hopper found an actual moth trapped in the Harvard Mark II computer\'s relay, causing a malfunction. Sh

In [14]:
#A different method for creating prompt template
#Passing a list instead of string with 
#this method basically merges prompt templates and LLM messages in a single method
template = ChatPromptTemplate([
        ("system", "You are a helpful assistant that tells interesting facts."),
        ("human", "Tell me about {topic}")
    ])
chain = template | llm
response = chain.invoke({"topic": "programming"})
print(response)

content='That\'s a fantastic topic! Programming is truly the hidden language of our modern world. Here are a few interesting facts about it:\n\n1.  **It\'s Like Writing a Recipe for a Computer:** At its core, programming is about writing a set of instructions, called "code," that tells a computer exactly what to do. Think of it like a super detailed recipe: if you miss a step or get it wrong, the computer won\'t know what to do, or it will produce something unexpected!\n\n2.  **The First Programmer Was a Woman:** The very first computer programmer is widely considered to be **Ada Lovelace**, the daughter of the poet Lord Byron. Back in the mid-1800s, she wrote what\'s often recognized as the first algorithm intended to be carried out by a machine – Charles Babbage\'s analytical engine. She saw the potential for computers to do far more than just calculations!\n\n3.  **There Are Hundreds of Programming Languages:** Just like humans speak different languages, computers understand differe

In [16]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

def load_document(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents
folder_path = "C:/Users/ladha/Machine Learning/RAG Chatbot/docs"

documents = load_document(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
split = text_splitter.split_documents(documents)
print(f"Split the documents into {len(split)} chunks")

Loaded 5 documents from the folder.
Split the documents into 41 chunks


In [17]:
split[0]

Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-09-21T21:41:12+05:30', 'author': 'JASH LADHANI - 70322100155', 'moddate': '2025-09-21T21:41:12+05:30', 'source': 'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\amazon_report.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Letter from the CEO \nDear Shareholders, \n2023 was a year of transformation and growth for Amazon. We continued to innovate on behalf of \nour customers while making significant investments in emerging technologies, sustainability, and \nglobal expansion. Our relentless focus on customer obsession, long-term thinking, and operational \nexcellence has enabled us to deliver strong results across all our business segments. \nWe expanded our leadership in cloud computing, revolutionized logistics and delivery, and pioneered \nnew frontiers in artificial intelligence and machine learning. Our commitment to being Earth's Most \nCustomer-C

In [19]:
print(split[0].metadata)
print(split[0].page_content)

{'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-09-21T21:41:12+05:30', 'author': 'JASH LADHANI - 70322100155', 'moddate': '2025-09-21T21:41:12+05:30', 'source': 'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\amazon_report.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
Letter from the CEO 
Dear Shareholders, 
2023 was a year of transformation and growth for Amazon. We continued to innovate on behalf of 
our customers while making significant investments in emerging technologies, sustainability, and 
global expansion. Our relentless focus on customer obsession, long-term thinking, and operational 
excellence has enabled us to deliver strong results across all our business segments. 
We expanded our leadership in cloud computing, revolutionized logistics and delivery, and pioneered 
new frontiers in artificial intelligence and machine learning. Our commitment to being Earth's Most 
Customer-Centric Company remains unwavering as we b

In [21]:
#Now we create embeddings for the RAG Systems

#We canuse either google gemini embeddings or Transformers
#from langchain_google_genai import GoogleGenerativeAIEmbeddings
#1. Gemini embeddings
#embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#document_embeddings = embeddings.embed_documents([split.page_content for split in split])
#print(f"Created embeddings for {len(document_embeddings)} document chunks")

In [22]:
#Using sentence transformers
#!pip install -U langchain-huggingface

In [23]:
embedding_function = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in split])
document_embeddings[0]

[0.02229335904121399,
 -0.09387563169002533,
 0.06080594286322594,
 -0.040375541895627975,
 0.04493541270494461,
 -0.023383354768157005,
 -0.02749449387192726,
 -0.0476890429854393,
 0.07645419985055923,
 -0.0016002620104700327,
 -0.010110734961926937,
 0.0703221932053566,
 0.04522973671555519,
 -0.049691181629896164,
 0.005457814317196608,
 -0.04428117349743843,
 -0.04753728583455086,
 -0.14613069593906403,
 -0.10447579622268677,
 -0.07812505960464478,
 -0.057936765253543854,
 0.037363916635513306,
 -0.001877667848020792,
 0.04227926582098007,
 -0.08310526609420776,
 0.10157390683889389,
 -0.022628696635365486,
 0.03692926838994026,
 -0.026070842519402504,
 -0.0392434298992157,
 0.025050614029169083,
 0.0031171604059636593,
 0.11538727581501007,
 0.05133163556456566,
 -0.06464819610118866,
 0.01940147392451763,
 -0.0559970960021019,
 -0.07019898295402527,
 0.030206289142370224,
 -0.00016518098709639162,
 0.0438443198800087,
 -0.03930535167455673,
 -0.03678524121642113,
 -0.00257084891

In [27]:
#Now we store these embeddings

#We use ChromaDB as VectorDB

from langchain_chroma import Chroma
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name = collection_name, documents=split, embedding=embedding_function, persist_directory="./chroma_db")

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [28]:
#Similarity search

#Through this we get matching documents
query = "When was amazon founded?"
search_results = vectorstore.similarity_search(query, k=2)

print(f"Top 2 relevent chunks for the query: '{query}'\n")
for i, result in enumerate(search_results,1):
    print(f"Result {i}: ")
    print(f"Source: {result.metadata.get('source','unknown')}")
    print(f"Content: {result.page_content}")
    print()

Top 2 relevent chunks for the query: 'When was amazon founded?'

Result 1: 
Source: C:/Users/ladha/Machine Learning/RAG Chatbot/docs\amazon_report.pdf
Content: Letter from the CEO 
Dear Shareholders, 
2023 was a year of transformation and growth for Amazon. We continued to innovate on behalf of 
our customers while making significant investments in emerging technologies, sustainability, and 
global expansion. Our relentless focus on customer obsession, long-term thinking, and operational 
excellence has enabled us to deliver strong results across all our business segments. 
We expanded our leadership in cloud computing, revolutionized logistics and delivery, and pioneered 
new frontiers in artificial intelligence and machine learning. Our commitment to being Earth's Most 
Customer-Centric Company remains unwavering as we build for the next decade of growth. 
Company Overview 
Amazon.com, Inc. is a multinational technology company focusing on e-commerce, cloud computing, 
digital stream

In [29]:
#We cant store this vectorstore output inside the chain, so we need to convert it to the retriever object
retriever = vectorstore.as_retriever(search_kwargs={"k":2})
retriever.invoke("When was jpmorgan founded?")

[Document(id='c736e426-2b9e-4321-9e80-96a293165573', metadata={'source': 'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\JPMorgan_Annual_Report.docx'}, page_content="JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. 

In [30]:
#Add the retriever to the chain
#We will first create a template and prompt for our model to work perfectly
from langchain_core.prompts import ChatMessagePromptTemplate
template = """Answer the question based on the following context:
{context}

Question: {question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

In [31]:
#We can build our chain and also use langchain's pre built chain
#We will first build our own chain

#In our chain, we create a dictionary and pass to the prompt so that the model can dynamically take input questions and context
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt    #RunnablePassThrough is basically what we pss in invoke
)
rag_chain.invoke("Asset values of JP Morgan")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based on the following context:\n[Document(id=\'7af4b431-a6dc-484d-b85c-1666b9381489\', metadata={\'source\': \'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\\\JPMorgan_Annual_Report.docx\'}, page_content="assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world\'s most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world\'s most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world\'s most prominent corporate, institutional, and government clients."), Document(id=\'a3bcf86a-b06c-4b70-a138-ed13a5a6d0ce\', met

In [32]:
#Currently the model provides extra info while we only need the content

def doc2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)  #fetches texts for the selected documents and joins them and seperates using \n\n

In [33]:
#we update the chain to pass the retriever through this new function that we created\
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever | doc2str, "question": RunnablePassthrough()} | prompt    #RunnablePassThrough is basically what we pss in invoke
)
rag_chain.invoke("Asset values of JP Morgan")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based on the following context:\nassets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients.\n\nassets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm

In [34]:
#Now we add llm to the chain
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever | doc2str, "question": RunnablePassthrough()} #RunnablePassThrough is basically what we pss in invoke
    | prompt
    | llm
    | StrOutputParser()
)
question = "Asset values of JP Morgan"
response = rag_chain.invoke(question)
print(response)

JPMorgan Chase & Co. has assets exceeding $3.9 trillion.


In [35]:
#So now the model answers the question but it won't be able to answer followup questions since it is not a conversational model

In [36]:
#For conversational RAG

#First we introduce the chat history
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    SystemMessage(content=response)
])

In [37]:
chat_history

[HumanMessage(content='Asset values of JP Morgan', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='JPMorgan Chase & Co. has assets exceeding $3.9 trillion.', additional_kwargs={}, response_metadata={})]

In [38]:
#To handle followup questions we contextualize our system
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.

"""
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "Where is it's headquaters?", "chat_history":chat_history})

"Where is JPMorgan Chase & Co.'s headquarters?"

In [39]:
#Above o/p is right, it doesnt provide answer yet, it was only required to contextualize model

In [40]:
#We will now make our model history aware
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt                           #to create history aware retriever, it uses our llm and normal retriever that we made above and the contextualize_q_prompt which we used to make our model understand context
)
history_aware_retriever.invoke({"input":"Where is it's headquarter?", "chat_history":chat_history})

[Document(id='a3bcf86a-b06c-4b70-a138-ed13a5a6d0ce', metadata={'source': 'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\JPMorgan_Annual_Report.docx'}, page_content="assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients."),
 Document(id='11a9eca2-dcfb-40a6-84c0-1e54353c77d5', metadata={'source': 'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\JPMorgan_Annual_Report.docx'}, page_content="

In [41]:
#Now we create a final chain to join everything to create final llm
#Creating another prompt to adapt to above updates
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI assistant. Use the following context to answer the user's question"),
    ("system","Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
rag_chain.invoke({"input":"How many customers in the United States do they serve?", "chat_history":chat_history})

{'input': 'How many customers in the United States do they serve?',
 'chat_history': [HumanMessage(content='Asset values of JP Morgan', additional_kwargs={}, response_metadata={}),
  SystemMessage(content='JPMorgan Chase & Co. has assets exceeding $3.9 trillion.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='ca522cea-31fe-42fe-bc03-70e736d30195', metadata={'source': 'C:/Users/ladha/Machine Learning/RAG Chatbot/docs\\JPMorgan_Annual_Report.docx'}, page_content="assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 trillion. We serve millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients. JPMorgan Chase & Co. is a leading global financial services firm with assets exceeding $3.9 

In [43]:
#Using SQLite database to store everything
import sqlite3
from datetime import datetime
DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row     #Normally sql queries return tuples, to make it more readable we do this. This converts the queries to a dictionary 
    return conn
def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )'''
    )
    conn.close()

#function to capture session id, what user asked, what response and he got and then it will save it inside the above table 
def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

#To get chat history for a particular session
def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at',(session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role":"human", "content":row['user_query']},
            {"role":"ai", "content":row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [45]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
q1 = "when was amazon founded?"
a1 = rag_chain.invoke({"input":q1,"chat_history":chat_history})['answer']
insert_application_logs(session_id, q1, a1, "gemini-2.5-flash")
print(f"Human: {q1}")
print(f"AI: {a1}")

[]
Human: when was amazon founded?
AI: Amazon was founded in **1994**.


In [54]:
q2 = "Give a short summary on it's features?"
chat_history = get_chat_history(session_id)
a2 = rag_chain.invoke({"input": q2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, q2, a2, "gemini-2.5-pro")
print(f"Human: {q2}")
print(f"AI: {a2}")

Human: Give a short summary on it's features?
AI: Amazon.com, Inc. is a multinational technology company primarily focused on **e-commerce, cloud computing (AWS), digital streaming, and artificial intelligence**. Founded in 1994, it operates globally in over 180 countries, serving hundreds of millions of customers and employing over 1.5 million people. Its mission is to be Earth's Most Customer-Centric Company, Best Employer, and Safest Place to Work. Its business is divided into **North America and International segments**, both handling retail sales of consumer products and subscriptions through online stores, including export sales.


In [56]:
#For new user
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
q1 = "Asset value of JP Morgan bank?"
a1 = rag_chain.invoke({"input":q1,"chat_history":chat_history})['answer']
insert_application_logs(session_id, q1, a1, "gemini-2.5-flash")
print(f"Human: {q1}")
print(f"AI: {a1}")

[]
Human: Asset value of JP Morgan bank?
AI: JPMorgan Chase & Co. has assets exceeding $3.9 trillion.


In [58]:
q2 = "How many customers do they have in the US?"
chat_history = get_chat_history(session_id)
a2 = rag_chain.invoke({"input": q2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, q2, a2, "gemini-2.5-pro")
print(f"Human: {q2}")
print(f"AI: {a2}")

Human: How many customers do they have in the US?
AI: JPMorgan Chase & Co. serves millions of customers in the United States.


In [4]:
#Now we are done with RAG LLM
#Now we will make it production ready and deploy using FastAPI

#We will basically be creating an API
#We will convert our collab notebook into an API
#Learn how any frontend ->streamlit react.js etc communicate with our API
#Also learn to manage session ids so that model is capable of answering followup questions for that particular session
